In [1]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 13.8 MB/s eta 0:00:00m eta 0:00:010:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.6 MB/s eta 0:00:00? eta -:--:--
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.37-cp310-cp310-manylinux_2_35_x86_64.whl size=2508657 sha256=83183ceff3f7ab8fe609642a69ee90ef5bea26a90ae6c2c9f6c69c41138f2bbb
  Stored in directory: /home/eelco/.cache/pip/wheels/5b/4f/36/1df917d3d21e5d04ce960616a6fb9e39c1548149620528a67c
Successfully built llama-cpp-python

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [5]:
from llama_cpp import Llama

llm = Llama(
    model_path="../models/towerinstruct-7b-v0.1.Q4_K_M.gguf",
    n_ctx=4096,
    n_threads=8,
    n_gpu_layers=0,
    chat_format="chatml",
)


llama_model_loader: loaded meta data with 25 key-value pairs and 291 tensors from ../models/towerinstruct-7b-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_

In [19]:
QUESTION_SYSTEM_PROMPT="""
You are an assistant that translates between the requested languages.
- If your translations contain code, only comments are translated.
- API calls and code should be left untouched to remain functional

Example:
user: translate en to fr: Where do I find the Twitter Activity Logs API?
assistant: Où puis-je trouver l'API des journaux d'activité Twitter?
"""


# llm.create_chat_completion(
#     messages = [
#         {"role": "system", "content": SYSTEM_PROMPT},
#         {
#             "role": "user",
#             "content": "translate en to fr: How many people from the Netherlands are active on X?"
#         }
#     ]
# )

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3832.14 ms
llama_print_timings:      sample time =       4.54 ms /    16 runs   (    0.28 ms per token,  3528.11 tokens per second)
llama_print_timings: prompt eval time =     752.42 ms /    22 tokens (   34.20 ms per token,    29.24 tokens per second)
llama_print_timings:        eval time =    2316.44 ms /    15 runs   (  154.43 ms per token,     6.48 tokens per second)
llama_print_timings:       total time =    3105.19 ms /    37 tokens


{'id': 'chatcmpl-2872fb26-78c3-491d-80f6-b3ee03044c8d',
 'object': 'chat.completion',
 'created': 1706696501,
 'model': '../models/towerinstruct-7b-v0.1.Q4_K_M.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Combien de personnes des Pays-Bas sont actives sur X ?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 120, 'completion_tokens': 15, 'total_tokens': 135}}

In [66]:
from data import get_training_data
import langdetect

questions, answers, _ = get_training_data()

languages = [langdetect.detect(q) for q in questions]

In [93]:
print(answers[1][0])

L'API "Ad Library" de Meta permet de connaître la cible prévue pour une campagne publicitaire, et permet d'obtenir des informations démographiques sur les personnes ayant été effectivement touchées par cette campagne. La documentation de cette API se trouve à l'adresse suivante : https://www.facebook.com/ads/library/api/. Voici quelques champs pertinents :
- 'target_ages', 'target_gender', 'target_locations' : ces champs permettent de comprendre la cible prévue pour la campagne.
- 'impressions', 'eu_total_reach', 'estimated_audience_size', 'demographic_distribution', 'age_country_gender_reach_breakdown' : ces champs permettent d'évaluer la performance de la campagne et le profil socio-démographique des personnes ayant interagi avec cette publicité. Certains champs sont soumis à conditions (publicité politique, publicité ayant visé un ou plusieurs pays européens).



In [94]:
translated_qa = []

def translate_qa(q, src_language, tgt_language):
    answer = llm.create_chat_completion(
        messages = [
            # {"role": "system", "content": QUESTION_SYSTEM_PROMPT},
            {
                "role": "user",
                "content": f"Translate from {src_language} to {tgt_language}. Your response should ONLY contain the translation, no other comments or answers. \n\n {q}\n\n Translation:"
            }
        ]
    )
    return answer["choices"][0]["message"]["content"]


print(answers[1][0])
print()
print(translate_qa(answers[1][0], "fr", "en"))

# qa_translated = []
# for q, a, lang in zip(questions, answers, languages):
#     a = a[0]
#     if lang == "en":
#         continue
#     print("======")
#     # print(q)
#     tq = translate_qa(q, lang, "en")
#     print(tq)
#     print("-----")
#     # print(a)
#     ta = translate_qa(a, lang, "en")
#     print(ta)
    
#     qa_translated.append((tq, ta))

L'API "Ad Library" de Meta permet de connaître la cible prévue pour une campagne publicitaire, et permet d'obtenir des informations démographiques sur les personnes ayant été effectivement touchées par cette campagne. La documentation de cette API se trouve à l'adresse suivante : https://www.facebook.com/ads/library/api/. Voici quelques champs pertinents :
- 'target_ages', 'target_gender', 'target_locations' : ces champs permettent de comprendre la cible prévue pour la campagne.
- 'impressions', 'eu_total_reach', 'estimated_audience_size', 'demographic_distribution', 'age_country_gender_reach_breakdown' : ces champs permettent d'évaluer la performance de la campagne et le profil socio-démographique des personnes ayant interagi avec cette publicité. Certains champs sont soumis à conditions (publicité politique, publicité ayant visé un ou plusieurs pays européens).




Llama.generate: prefix-match hit


The Meta Ad Library API allows you to know the intended target for an advertising campaign and obtain demographic information about people who have actually been reached by this campaign. The documentation for this API can be found at the following address: https://www.facebook.com/ads/library/api/. Here are some relevant fields:
- 'target_ages', 'target_gender', 'target_locations': these fields allow you to understand the intended target for the campaign. - 'impressions', 'eu_total_reach', 'estimated_audience_size', 'demographic_distribution', 'age_country_gender_reach_breakdown': these fields allow you to evaluate the performance of the campaign and the socio-demographic profile of people who have interacted with this advertising. Some fields are subject to conditions (political advertising, advertising targeting one or more European countries).



llama_print_timings:        load time =    3832.14 ms
llama_print_timings:      sample time =      55.56 ms /   193 runs   (    0.29 ms per token,  3473.66 tokens per second)
llama_print_timings: prompt eval time =    7786.75 ms /   241 tokens (   32.31 ms per token,    30.95 tokens per second)
llama_print_timings:        eval time =   31612.35 ms /   192 runs   (  164.65 ms per token,     6.07 tokens per second)
llama_print_timings:       total time =   39889.12 ms /   433 tokens


In [ ]:
print(qa_translated)

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024, chunk_overlap=0
)


docs = TextLoader("../data/platform-docs-versions/Snapchat_Transparency-World/Transparency Report.md").load()
from IPython.display import display, Markdown
# display(Markdown(doc[0].page_content))

split = text_splitter.split_documents(docs)

In [42]:
print(split[0].page_content)

# Resource URL: https://values.snap.com/fr-FR/privacy/transparency
Rapport de transparence

1er janvier 2023 - 30 juin 2023

Publication :

25 octobre 2023

Mise à jour :

25 octobre 2023

Pour donner un aperçu des efforts de Snap en matière de sécurité ainsi que de la nature et du volume du contenu signalé sur notre plateforme, nous publions des rapports sur la transparence deux fois par an. Nous entendons poursuivre la publication de ces rapports dans l'objectif de les rendre plus complets et plus documentés, afin que les nombreuses parties prenantes qui s'intéressent de près à nos pratiques en matière de modération de contenu et d'application de la loi, ainsi qu'au bien-être de notre communauté puissent en profiter.


In [49]:
SYSTEM_PROMPT="""
You are an assistant that translates technical markdown documentation from french to english.
- French documentation is fed to you in chunks, the only thing you answer is a english translation
- Subsequent chunks are all from the same document
- If the provided contains just random characters (like an embedded image), you omit this part.
"""

def translate(split, chat_log=None):
    if not chat_log:
        chat_log = [{"role": "system", "content": SYSTEM_PROMPT}]
    chat_log.append({"role": "user", "content": split})
    answer = llm.create_chat_completion(
        messages=chat_log
    )
    answer = answer["choices"][0]["message"]
    chat_log.append(answer)
    return chat_log
    

In [50]:
chat_log = None
chat_log = translate(split[0].page_content, chat_log)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3832.14 ms
llama_print_timings:      sample time =      39.05 ms /   136 runs   (    0.29 ms per token,  3483.07 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   22159.14 ms /   136 runs   (  162.93 ms per token,     6.14 tokens per second)
llama_print_timings:       total time =   22490.46 ms /   137 tokens


In [55]:
chat_log = None
translations = []
for i in range(3):
    chat_log = translate(split[i].page_content, chat_log)
    translations.append(chat_log[-1]["content"])

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3832.14 ms
llama_print_timings:      sample time =      44.10 ms /   152 runs   (    0.29 ms per token,  3446.63 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   24288.03 ms /   152 runs   (  159.79 ms per token,     6.26 tokens per second)
llama_print_timings:       total time =   24666.91 ms /   153 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3832.14 ms
llama_print_timings:      sample time =      29.07 ms /    99 runs   (    0.29 ms per token,  3405.22 tokens per second)
llama_print_timings: prompt eval time =    4967.88 ms /   152 tokens (   32.68 ms per token,    30.60 tokens per second)
llama_print_timings:        eval time =   16911.61 ms /    98 runs   (  172.57 ms per token,     5.79 tokens per second)
llama_print_timings:       to

In [59]:
print("\n".join(translations))

Transparency Report

January 1st 2023 - June 30th 2023

Publication:
October 25th 2023

Update:
October 25th 2023

To give an overview of Snap's security efforts as well as the nature and volume of content reported on our platform, we publish transparency reports twice a year. We aim to continue publishing these reports with more comprehensive and documented information so that the many stakeholders who are closely interested in our content moderation practices and enforcement, as well as the well-being of our community can benefit from them.
This transparency report covers the first half of 2021 (January 1st - June 30th). As with our previous reports, we are sharing data on the total number of content in the app, the account level reports we received and acted upon in specific categories of policy violations, how we responded to law enforcement and government requests, and our various sanctions broken down by country. And our enforcement actions broken down by country.
As part of our 